In [0]:
# Installing necessary packages for database connection, Azure services, and data processing
%pip install psycopg2
%pip install adlfs
%pip install azure-identity
%pip install azure-keyvault-secrets
%pip install openpyxl
%pip install azure-identity azure-storage-blob
%pip install azure-storage-blob azure-mgmt-datafactory
%pip install azure-monitor-query
%pip install azure-identity azure-mgmt-monitor



In [0]:
# Set and display the environment widget for selecting the deployment environment
dbutils.widgets.text("environment", "qa")
env = dbutils.widgets.get("environment")
display(f"environment : {env}")

In [0]:
# Utility functions and configuration for connecting to Azure services, executing SQL queries, reading files, and managing outputs for data pipeline testing and validation in Databricks

import os
import sys
import logging
import psycopg2
import pandas.io.sql as sqlio
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from pyspark.sql import SparkSession
sys.path.append(f'{os.getcwd()}/')
import constants
from azure.storage.blob import BlobServiceClient
from azure.mgmt.datafactory import DataFactoryManagementClient
from azure.mgmt.datafactory.models import RunQueryFilter, RunQueryFilterOperand, RunQueryFilterOperator, RunFilterParameters
from datetime import datetime, timedelta, timezone
import requests
import importlib
import constants
import pandas as pd
import json

importlib.reload(constants)
DB_CONFIG = {
    "user": constants.user,
    "host": constants.host,
    "port": constants.port,
    "database": constants.database.format(env = env),
}

subscription_id = constants.subscription_id
key_vault_name = constants.key_vault_name
resource_group_name = constants.resource_group_name
factory_name = constants.factory_name
dap_nonprod_sql_pwd_nm = constants.dap_nonprod_sql_pwd_nm
dap_nonprod_emailer_URL_nm = constants.dap_nonprod_emailer_URL_nm
DAP_Orchestration_Pipeline = constants.DAP_Orchestration_Pipeline.format(env=env)
FAV_Pipeline = constants.FAV_Pipeline.format(env=env)
DVP_Pipeline = constants.DVP_Pipeline.format(env=env)
ADF_run_from_feed_master_query = constants.ADF_run_from_feed_master_query


# -----------------------------------
# Helpers
# -----------------------------------

def get_secret(secret_name, key_vault_name):
    credential = DefaultAzureCredential()
    client = SecretClient(vault_url=f"https://{key_vault_name}.vault.azure.net/", credential=credential)
    return client.get_secret(secret_name).value

def validate_email(actual_email,template: dict):
    for keyword in template.get('keywords',[]):
        print(keyword)
        if not actual_email.str.contains(keyword, case=False, na=False).any():
            return False
    return True


def get_db_connection():
    password = get_secret(dap_nonprod_sql_pwd_nm, key_vault_name)
    return psycopg2.connect(
        user=DB_CONFIG["user"],
        password=password,
        host=DB_CONFIG["host"],
        port=DB_CONFIG["port"],
        database=DB_CONFIG["database"]
    )

def execute_query(query):
    try:
        with get_db_connection() as conn:
            df = sqlio.read_sql_query(query, conn)
            return df
    except Exception as e:
        display(f"Query execution failed: {e}")
        raise

def read_sql_file(file_path):
    if not os.path.exists(file_path):
        display(f"SQL file not found: {file_path}")
        return None
    with open(file_path, 'r') as file:
        return file.read()

# -----------------------------------
# Core Functionalities
# -----------------------------------

def run_post_test_sql(sql_file_path, test_id):
    query = read_sql_file(sql_file_path)
    if not query:

        return "FAIL"
    try:
        display(query)
        df = execute_query(query)
        if df.success_flag[0] == 1:
            return "PASS"
        else:
            return "FAIL"
    except Exception as e:
        display(f"Error in post-test SQL for {test_id}: {e}")
        return "FAIL"

def get_rerun_count(query):
    try:
        return execute_query(query)
    except Exception as e:
        display(f"Failed to get rerun count: {e}")
        return None

def get_error_result_from_sql(sql_file_path):
    query = read_sql_file(sql_file_path)
    error_message = ""
    if not query:
        return None
    try:
        df = execute_query(query)
        if df.empty:
            error = "SQL result is empty"
        elif 'error_message' not in df.columns:
            error = "'error_message' column missing in SQL result"
        else:
            raw_error = df.iloc[0]['error_message']
            if raw_error and len(raw_error) > 0:
                error = raw_error[0]
            else:
                error = "Error message is empty or invalid"
        return error
    except Exception as e:
        display(f"Failed to fetch error result: {e}")
        return None

# -----------------------------------
# Parquet/CSV Read Utilities
# -----------------------------------

def read_parquet_count_rows(file_path):
    df = spark.read.parquet(file_path)
    return df.count()

def read_parquet_file_count(file_path):
    df = spark.read.parquet(file_path)
    return len(df.inputFiles())

def read_source_row_count(file_path):
    if not file_path.startswith("file:/"):
        file_path = f"file://{file_path}"
    if file_path.endswith('.csv') or file_path.endswith('.txt') or file_path.endswith('.dat'):
        df = spark.read.csv(file_path, header=True)
        return df.count()
# -----------------------------------
# Output Management
# -----------------------------------

outputs = {}

def save_output(test_id, description, result, error="null"):
    outputs[test_id] = {
        "description": description,
        "result": result,
        "error": error
    }

def get_pipeline_runs(subscription_id, resource_group_name, factory_name, pipeline_name, minutes=120):
    credential = DefaultAzureCredential()
    adf_client = DataFactoryManagementClient(credential, subscription_id)
    end_time = datetime.now(timezone.utc)
    start_time = end_time - timedelta(minutes=minutes)
    
    target_pipeline_name = pipeline_name.format(env = env)
    display(f"Target pipeline name: {target_pipeline_name}")
    
    pipeline_name_filter = RunQueryFilter(
        operand=RunQueryFilterOperand.PIPELINE_NAME,
        operator=RunQueryFilterOperator.EQUALS,
        values=[target_pipeline_name]
    )
    
    filter_params = RunFilterParameters(
        last_updated_after=start_time,
        last_updated_before=end_time,
        filters=[pipeline_name_filter]
    )
    
    paged_pipeline_runs = adf_client.pipeline_runs.query_by_factory(
        resource_group_name=resource_group_name,
        factory_name=factory_name,
        filter_parameters=filter_params
    )
    
    return paged_pipeline_runs


def get_azure_token(resource="https://management.azure.com/"):
    url = "http://169.254.169.254/metadata/identity/oauth2/token"
    params = {
        "api-version": "2018-02-01",
        "resource": resource
    }
    headers = {
        "Metadata": "true"
    }
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()["access_token"]

    
def fetch_today_logic_app_runs(subscription_id, resource_group, logic_app_name , count_of_rows = 250):
    token = get_azure_token()
    url = (
        f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{resource_group}"
        f"/providers/Microsoft.Logic/workflows/{logic_app_name}/runs"   
    )
    params = {
        "api-version": "2016-06-01",
        "$top": count_of_rows 
    }
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    response = requests.get(url, headers=headers , params = params)
    response.raise_for_status()
    all_runs = response.json().get('value')
    # Filter today runs 
    today_utc_date = datetime.now(timezone.utc).date()
    today_runs = [
        run for run in all_runs
        if datetime.fromisoformat(run['properties']['startTime'].replace('Z', '+00:00')).date() == today_utc_date
    ]
    # Prepare rows
    rows = []
    for run in today_runs:
        tracking_id = run['name']
        uri = run['properties']['trigger']['outputsLink']['uri']
        response = requests.get(uri)
        response_body = response.json().get('body')
        rows.append((tracking_id, uri, json.dumps(response_body)))  # Convert response to string
    # Create Spark DataFrame
    df = spark.createDataFrame(rows, ["tracking_id", "uri", "response"])
    return df

def trigger_azure_function(function_app_name, function_name, master_key, input_payload=""):
    """
    Triggers an Azure Function via HTTP POST to the admin endpoint.

    Args:
        function_app_name (str): Name of the deployed Azure Function App.
        function_name (str): Name of the function to trigger.
        master_key (str): Master key for authentication.
        input_payload (str or dict): Payload to send as 'input' (default: empty string).

    Returns:
        dict: Result with status and response text.
    """
    print(f"{function_app_name} \n {function_name} \n {master_key}")
    admin_url = f"https://{function_app_name}.azurewebsites.net/admin/functions/{function_name}"
    headers = {
        "Content-Type": "application/json",
        "x-functions-key": master_key
    }
    data = {
        "input": input_payload
    }
    try:
        response = requests.post(admin_url, headers=headers, data=json.dumps(data))
        return {
            "status_code": response.status_code,
            "response_text": response.text
        }
    except requests.exceptions.RequestException as e:
        return {
            "status_code": None,
            "error": str(e)
        }
        

In [0]:
# Fetching pipeline runs for the last 120 minutes for DAP Orchestration, DVP, and FAV pipelines
DAP_Orchestration_runs = get_pipeline_runs(subscription_id , resource_group_name, factory_name, DAP_Orchestration_Pipeline, minutes=120)
DVP_runs = get_pipeline_runs(subscription_id , resource_group_name, factory_name, DVP_Pipeline, minutes=120)
FAV_runs = get_pipeline_runs(subscription_id , resource_group_name, factory_name, FAV_Pipeline, minutes=120)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf , udf, col
from pyspark.sql.types import StringType, BooleanType
import requests
import json

# Fetch logic app runs
df = fetch_today_logic_app_runs(constants.logic_app_subscription_id, constants.logic_app_resource_group, constants.logic_app_name)

# Define UDFs for filtering
def contains_keyword(keyword):
    return udf(lambda x: keyword in x if x else False, BooleanType())

def contains_all_keywords(keywords):
    return udf(lambda x: all(k in x for k in keywords) if x else False, BooleanType())

# Apply filters
filtered_df = df.filter(contains_all_keywords(["centura","autotest"])(col("response")))

incomplete_feedname_df = filtered_df.filter(
    contains_keyword("Incomplete for feed name")(col("response"))
).limit(1)

complete_feedname_df = filtered_df.filter(
    contains_keyword("Complete for feed name")(col("response"))
).limit(1)

adf_failure_df = filtered_df.filter(
    contains_keyword("ADF Process Failure Notification")(col("response"))
).limit(1)

# Format JSON response (optional pretty print)
def pretty_json(x):
    try:
        return json.dumps(json.loads(x), indent=2)
    except:
        return x
pretty_json_udf = udf(pretty_json, StringType())

incomplete_feedname_df = incomplete_feedname_df.withColumn("response", pretty_json_udf(col("response")))
complete_feedname_df = complete_feedname_df.withColumn("response", pretty_json_udf(col("response")))
adf_failure_df = adf_failure_df.withColumn("response", pretty_json_udf(col("response")))

# Filter for FAM emails
FAM_Sub_Keywords = ['File Arrival Monitor', 'regressionv1']
FAM_emails_df = df.filter(contains_all_keywords(FAM_Sub_Keywords)(col("response"))).limit(1)

def validate_keywords_udf(keywords):
    return udf(lambda response: all(k.lower() in response.lower() for k in keywords), BooleanType())

# Define your keyword lists
incomplete_keywords = ["Incomplete for feed name", "centura_daily_autotest"]
complete_keywords = ["Complete for feed name", ]
adf_failure_keywords = ["ADF Process Failure Notification"]
FAM_keywords = ["File Arrival Monitor", "regressionv1"]

# Apply validation
incomplete_feedname_df = incomplete_feedname_df.withColumn(
    "is_valid", validate_keywords_udf(incomplete_keywords)(col("response"))
)

complete_feedname_df = complete_feedname_df.withColumn(
    "is_valid", validate_keywords_udf(complete_keywords)(col("response"))
)

adf_failure_df = adf_failure_df.withColumn(
    "is_valid", validate_keywords_udf(adf_failure_keywords)(col("response"))
)

FAM_emails_df = FAM_emails_df.withColumn(
    "is_valid", validate_keywords_udf(FAM_keywords)(col("response"))
)

# get counts for the mails triggered before manual trigger
feed_completion_count = filtered_df.filter(contains_keyword("Feed Completion Status")(col("response"))).count()
FAM_email_count = FAM_emails_df.count()

In [0]:
# Load test cases from the configuration file and filter active test cases into a dictionary
import json
import os
BasePath = os.path.dirname(os.getcwd())
RegressionPath = BasePath + "/regression/test_cases/"
ConfigPath = os.path.join(BasePath, "config/test_config.json")
with open(ConfigPath, 'r') as f:
    data = f.read()
jsonObject = json.loads(data)
test_cases = jsonObject.get('test_cases')
test_cases_dict = {}
test_cases_dict = {tc.get('TestID'): tc for tc in test_cases if tc.get('IsActive')=='true'}
display(test_cases_dict)

In [0]:
# A test case execution function for generic testcases
import pandas as pd
from openpyxl import load_workbook
def run_test_case(test_case, RegressionPath):
    test_id = test_case.get('TestID')
    id = test_case.get("ID")
    post_sql = test_case.get('PostTest_SQL')
    input_file = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    description = test_case.get('Test_Case_Description')
    error_sql = test_case.get("ErrorSQL")

    sql_path = f'{RegressionPath}/{post_sql}'
    display(sql_path)
    result = run_post_test_sql(sql_path, test_id)
    try:
        if result == "PASS":

            input_path = f'{RegressionPath}/tc{id}/input_files/{input_file}'
            if 'xlsx' in input_file:
                input_path = f'{RegressionPath}/tc{id}/input_files/{input_file}'
                wb = load_workbook(filename=input_path,data_only=True)
                sheet = wb.active
                source_count = sum(1 for row in sheet.iter_rows() if any(cell.value is not None for cell in row)) - 1 #as the excel file having header row
            else:
                source_count = read_source_row_count(input_path)
            parquet_file_count = read_parquet_file_count(Target_PB_Folder)
            parquet_row_count = read_parquet_count_rows(Target_PB_Folder)
            if parquet_file_count == 1 and source_count == parquet_row_count:
                display(f"Testcase {test_id} Passed")
                save_output(test_id, description, "PASS")
            else:
                error = get_error_result_from_sql(f'{RegressionPath}/{error_sql}')
                display(f"Testcase {test_id} Failed: {error}")
                save_output(test_id, description, "FAIL", error)

        else:
            display(f"Testcase {test_id} Failed due to post-test SQL")
            save_output(test_id, description, "FAIL", "Post-test SQL failed")
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        error = str(e)
        save_output(test_id, description, "FAIL", error)



In [0]:
test_case = test_cases_dict.get("TC_PUSH_001")
run_test_case(test_case, RegressionPath)

In [0]:
test_case = test_cases_dict.get("TC_PUSH_002")
run_test_case(test_case, RegressionPath)

In [0]:
test_case = test_cases_dict.get("TC_PUSH_003")
run_test_case(test_case, RegressionPath)

In [0]:
test_case = test_cases_dict.get("TC_PUSH_004")
run_test_case(test_case, RegressionPath)

In [0]:
test_case = test_cases_dict.get("TC_PUSH_005")
run_test_case(test_case, RegressionPath)

In [0]:
# This test case (TC_PUSH_005a) validates that two target Parquet partitions exist and counts their rows,
# ensuring the pipeline writes the expected number of output parts. It passes if both partitions exist and the post-test SQL passes.

test_case = test_cases_dict.get("TC_PUSH_005a")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder')
    test_case_desc = test_case.get('Test_Case_Description')
    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    error_sql = test_case.get("ErrorSQL")
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    # sql_result = run_sql_file(sql_file_path, post_test_sql, test_id)
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_005a:", sql_result)
    try:
        target_counts = {}
        for i in range(1, 3):
            target_path = Target_PB_Folder.format(env= env , part_count=i)
            target_counts[f'part{i}_count'] = read_parquet_count_rows(target_path)
        display(target_counts)
        if sql_result == 'PASS' and len(target_counts.keys()) == 2:
            display("Testcase TC_PUSH_005a Passed")
            save_output("TC_PUSH_005a",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_005a Failed")
            save_output("TC_PUSH_005a", test_case_desc,"FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
# This test case validates the row count consistency between source CSV files and target Parquet files for specified sources.

test_case = test_cases_dict.get("TC_PUSH_006")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder')
    test_case_desc = test_case.get('Test_Case_Description')
    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    error_sql = test_case.get("ErrorSQL")
    source_names = test_case.get('Sources')
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_006:", sql_result)
    try:
        if sql_result == "PASS":
            source_names = ['claims' , 'payers' , 'devices']
            source_count = {}
            for source in source_names: 
                file_path = f'file:{RegressionPath}/tc6/source_files/*{source}*'
                source_count[source] = spark.read.csv(file_path,header=True).count()
            display(source_count)

            target_count = {}
            for source in source_names: 
                target_path = Target_PB_Folder.format(env= env ,source=source)
                target_count[source] = read_parquet_count_rows(target_path)
            display(target_count)

            src_tgt_row_cnt_validation = True 

            if source_count.keys() == target_count.keys():
                for key in source_count:
                    if source_count[key] == target_count[key]:
                        pass
                    else:
                        src_tgt_row_cnt_validation = False
                        break
                    
        if sql_result == "PASS" and src_tgt_row_cnt_validation:
            display("Testcase TC_PUSH_006 Passed")
            save_output("TC_PUSH_006",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_006 Failed")
            save_output("TC_PUSH_006", test_case_desc,"FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
# This test case (TC_PUSH_009) validates that the target Parquet folder contains exactly one file and that the file has zero rows,
# ensuring that the pipeline correctly handles and writes empty datasets as expected.

test_case = test_cases_dict.get("TC_PUSH_009")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    error_sql = test_case.get("ErrorSQL")
    test_case_desc = test_case.get('Test_Case_Description')  

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_009:", sql_result)
    try:
        if sql_result == "PASS":
            file_path = f'{RegressionPath}/tc9/input_files/{test_file_names}'

            # Read folder and count parquet files
            parquet_file_count = read_parquet_file_count(Target_PB_Folder)
            display("parquet_file_count: ", parquet_file_count)

            # Read parquet file and count rows
            parquet_row_count = read_parquet_count_rows(Target_PB_Folder)
            display("parquet_row_count: ", parquet_row_count)  

        if sql_result == "PASS" and parquet_file_count == 1 and parquet_row_count == 0:
            display("Testcase TC_PUSH_009 Passed")
            save_output("TC_PUSH_009",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_009 Failed")
            save_output("TC_PUSH_009", test_case_desc,"FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
# This test case (TC_PUSH_010) validates that the number of rows in the source input file matches the number of rows in the target Parquet file,
# and that exactly one Parquet file is generated in the target folder. The test passes if the SQL check passes, there is one Parquet file,
# and the row count in the source matches the row count in the target Parquet file.

test_case = test_cases_dict.get("TC_PUSH_010")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    error_sql = test_case.get("ErrorSQL")
    test_case_desc = test_case.get('Test_Case_Description')  

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_010:", sql_result)
    try:
        if sql_result == "PASS":
            file_path = f'{RegressionPath}/tc10/input_files/{test_file_names}'

            # Read source file with multiple extension support and count rows
            source_row_count = read_source_row_count(file_path)
            display("source_row_count: ", source_row_count)

            # Read folder and count parquet files
            parquet_file_count = read_parquet_file_count(Target_PB_Folder)
            display("parquet_file_count: ", parquet_file_count)

            # Read parquet file and count rows
            parquet_row_count = read_parquet_count_rows(Target_PB_Folder)
            display("parquet_row_count: ", parquet_row_count)  

        if sql_result == "PASS" and parquet_file_count == 1 and source_row_count == parquet_row_count:
            display("Testcase TC_PUSH_010 Passed")
            save_output("TC_PUSH_010",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_010 Failed")
            save_output("TC_PUSH_010",test_case_desc, "FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
# This test case (TC_PUSH_011) validates the pipeline by executing a post-test SQL check.
# The test passes if the SQL check returns "PASS". No file or row count validation is performed for this test case.
# The result and status are displayed and saved accordingly.

test_case = test_cases_dict.get("TC_PUSH_011")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    error_sql = test_case.get("ErrorSQL")
    test_case_desc = test_case.get('Test_Case_Description')  

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_011:", sql_result)
    try:
        if sql_result == "PASS":
            display("Testcase TC_PUSH_011 Passed")
            save_output("TC_PUSH_011",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_011 Failed")
            save_output("TC_PUSH_011", test_case_desc,"FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
# This test case (TC_PUSH_012) validates that the number of rows in the source input file matches the number of rows in the target Parquet file,
# and that exactly one Parquet file is generated in the target folder. The test passes if the SQL check passes, there is one Parquet file,
# and the row count in the source matches the row count in the target Parquet file.

test_case = test_cases_dict.get("TC_PUSH_012")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    error_sql = test_case.get("ErrorSQL")
    test_case_desc = test_case.get('Test_Case_Description')  

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_012:", sql_result)
    try:
        if sql_result == "PASS":
            file_path = f'{RegressionPath}/tc12/input_files/{test_file_names}'

            # Read source file with multiple extension support and count rows
            source_row_count = read_source_row_count(file_path)
            display("source_row_count: ", source_row_count)

            # Read folder and count parquet files
            parquet_file_count = read_parquet_file_count(Target_PB_Folder)
            display("parquet_file_count: ", parquet_file_count)

            # Read parquet file and count rows
            parquet_row_count = read_parquet_count_rows(Target_PB_Folder)
            display("parquet_row_count: ", parquet_row_count)  

        if sql_result == "PASS" and parquet_file_count == 1 and source_row_count == parquet_row_count:
            display("Testcase TC_PUSH_012 Passed")
            save_output("TC_PUSH_012", test_case_desc,"PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_012 Failed")
            save_output("TC_PUSH_012",test_case_desc, "FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
# This test case (TC_PUSH_013) validates the pipeline by executing a post-test SQL check.
# The test passes if the SQL check returns "PASS". No file or row count validation is performed for this test case.
# The result and status are displayed and saved accordingly.

test_case = test_cases_dict.get("TC_PUSH_013")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    test_case_desc = test_case.get('Test_Case_Description')
    error_sql = test_case.get("ErrorSQL")  

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)    
    display("Query execution status of TC_PUSH_013:", sql_result)
    try:
        if sql_result == "PASS":
            display("Testcase TC_PUSH_013 Passed")
            save_output("TC_PUSH_013",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_013 Failed")
            save_output("TC_PUSH_013",test_case_desc, "FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
# This test case (TC_PUSH_014) validates that the number of rows in the source input file matches the number of rows in the target Parquet file,
# and that exactly one Parquet file is generated in the target folder. The test passes if the SQL check passes, there is one Parquet file,
# and the row count in the source matches the row count in the target Parquet file. Additional verification for bad row data is pending.

test_case = test_cases_dict.get("TC_PUSH_014")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    test_case_desc = test_case.get('Test_Case_Description') 
    error_sql = test_case.get("ErrorSQL") 
    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_014:", sql_result)
    try:
        if sql_result == "PASS":
            file_path = f'{RegressionPath}/tc14/input_files/{test_file_names}'
            # Read source file with multiple extension support and count rows
            source_row_count = read_source_row_count(file_path)
            display("source_row_count: ", source_row_count)
            # Read folder and count parquet files
            parquet_file_count = read_parquet_file_count(Target_PB_Folder)
            display("parquet_file_count: ", parquet_file_count)
            # Read parquet file and count rows
            parquet_row_count = read_parquet_count_rows(Target_PB_Folder)
            display("parquet_row_count: ", parquet_row_count) 
        if sql_result == "PASS" and parquet_file_count == 1 and source_row_count == parquet_row_count:
            display("Testcase TC_PUSH_014 Passed")
            save_output("TC_PUSH_014",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_014 Failed")
            save_output("TC_PUSH_014",test_case_desc, "FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
# This test case (TC_PUSH_015) validates the pipeline by executing a post-test SQL check.
# The test passes if the SQL check returns "PASS". No file or row count validation is performed for this test case.
# The result and status are displayed and saved accordingly.

test_case = test_cases_dict.get("TC_PUSH_015")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    test_case_desc = test_case.get('Test_Case_Description') 
    error_sql = test_case.get("ErrorSQL") 

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_015:", sql_result)
    try:
        if sql_result == "PASS":
            file_path = f'{RegressionPath}/tc15/input_files/{test_file_names}'
        if sql_result == "PASS":
            display("Testcase TC_PUSH_015 Passed")
            save_output("TC_PUSH_015",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_015 Failed")
            save_output("TC_PUSH_015",test_case_desc, "FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
# This test case (TC_PUSH_016) validates that the number of rows in the source input file matches the number of rows in the target Parquet file,
# and that exactly one Parquet file is generated in the target folder. The test passes if the SQL check passes, there is one Parquet file,
# and the row count in the source matches the row count in the target Parquet file.

test_case = test_cases_dict.get("TC_PUSH_016")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    test_case_desc = test_case.get('Test_Case_Description') 
    error_sql = test_case.get("ErrorSQL") 

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_016:", sql_result)
    try:
        if sql_result == "PASS":
            file_path = f'{RegressionPath}/tc16/input_files/{test_file_names}'

            # Read source file with multiple extension support and count rows
            source_row_count = read_source_row_count(file_path)
            display("source_row_count: ", source_row_count)

            # Read folder and count parquet files
            parquet_file_count = read_parquet_file_count(Target_PB_Folder)
            display("parquet_file_count: ", parquet_file_count)

            # Read parquet file and count rows
            parquet_row_count = read_parquet_count_rows(Target_PB_Folder)
            display("parquet_row_count: ", parquet_row_count) 

        if sql_result == "PASS" and parquet_file_count == 1 and source_row_count == parquet_row_count:
            display("Testcase TC_PUSH_016 Passed")
            save_output("TC_PUSH_016",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_016 Failed")
            save_output("TC_PUSH_016", test_case_desc,"FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
test_case = test_cases_dict.get("TC_PUSH_017")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    test_case_desc = test_case.get('Test_Case_Description') 
    error_sql = test_case.get("ErrorSQL") 

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_017:", sql_result)
    try:

        if sql_result == "PASS":
            file_path = f'{RegressionPath}/{test_id}/input_files/{test_file_names}'

        if sql_result == "PASS":
            display("Testcase TC_PUSH_017 Passed")
            save_output("TC_PUSH_017",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_017 Failed")
            save_output("TC_PUSH_017",test_case_desc, "FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
test_case = test_cases_dict.get("TC_PUSH_018")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    test_case_desc = test_case.get('Test_Case_Description') 
    error_sql = test_case.get("ErrorSQL") 

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_018:", sql_result)
    try:
        if sql_result == "PASS":
            display("Testcase TC_PUSH_018 Passed")
            save_output("TC_PUSH_018",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_018 Failed")
            save_output("TC_PUSH_018", test_case_desc,"FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
test_case = test_cases_dict.get("TC_PUSH_019")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    test_case_desc = test_case.get('Test_Case_Description') 
    error_sql = test_case.get("ErrorSQL") 

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_019:", sql_result)
    try:
        if sql_result == "PASS":
            display("Testcase TC_PUSH_019 Passed")
            save_output("TC_PUSH_019",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_019 Failed")
            save_output("TC_PUSH_019",test_case_desc, "FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))


In [0]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.datafactory import DataFactoryManagementClient
from azure.mgmt.datafactory.models import RunFilterParameters
from datetime import datetime, timedelta
test_case = test_cases_dict.get("TC_PUSH_020")
test_id = test_case.get('TestID')
test_file_names = test_case.get('Test_File_Names')
Recipients = test_case.get('ConfigInternalEmailList')
test_case_desc = test_case.get('Test_Case_Description')
validation = False
try:
    for run in DVP_runs.value:
      if Recipients in str(run.parameters) and test_file_names in str(run.parameters):
        validation = True
        break
    if validation:
      save_output(test_id, test_case_desc, "PASS")
      display(f"Testcase {test_id} passed")
    else:
      error = f"{Recipients} not presented in DVP parameters"
      display(f"Testcase {test_id} failed")
      save_output(test_id, test_case_desc, "FAIL",error)
except Exception as e:  
  save_output(test_id, test_case_desc, "FAIL", str(e))
  display(f"Testcase {test_id} Failed due to exception: {e}")

In [0]:
# TC_PUSH_021: Trigger Feed Completion Function App and validate response
test_case_21 = test_cases_dict.get("TC_PUSH_021")
if test_case_21:
    test_case_desc_21 = test_case_21.get('Test_Case_Description')
    test_id_21 = test_case_21.get('TestID')
    display(f"{test_id_21} : {test_case_desc_21}")
    master_key = get_secret(constants.Function_App_MasterKey_Secret_Name, key_vault_name)
    Feed_Completion_response = trigger_azure_function(constants.function_app_name, constants.Feed_completion_function_name, master_key)
    if Feed_Completion_response.get("status_code") == 202:
        df = fetch_today_logic_app_runs(constants.logic_app_subscription_id, constants.logic_app_resource_group, constants.logic_app_name)
        Updated_feed_completion_count = df.filter(validate_keywords_udf(["Feed Completion Status","centura","autotest"])(col("response"))).count()
        if Updated_feed_completion_count > feed_completion_count:
            display("TC_PUSH_021 Passed")
            save_output("TC_PUSH_021", test_case_desc_21, "PASS")
        else:
            display("TC_PUSH_021 Failed")
            save_output("TC_PUSH_021", test_case_desc_21, "FAIL", f"no mail received after the manual trigger for feed completion ")
    else:
        display("TC_PUSH_021 Failed")
        save_output("TC_PUSH_021", test_case_desc_21, "FAIL", f"Not able to trigger the Function App {constants.Feed_completion_function_name}")

In [0]:
test_case = test_cases_dict.get("TC_PUSH_022")
test_id = test_case.get('TestID')
id = test_case.get("ID")
post_sql = test_case.get('PostTest_SQL')
input_file = test_case.get('Test_File_Names')
Target_PB_Folder = test_case.get('Target_PB_Folder')
description = test_case.get('Test_Case_Description')
error_sql = test_case.get("ErrorSQL")
sql_path = f'{RegressionPath}/{post_sql}'
display(sql_path)
result = run_post_test_sql(sql_path, test_id)
try:
    if result == "PASS":
        counts = {key: read_parquet_count_rows(path.format(env=env)) for key, path in Target_PB_Folder.items()}
        display(counts)
        if all(count > 0 for count in counts.values()):
            display(f"Testcase {test_id} Passed")
            save_output(test_id, description, "PASS")          
        else:
            error = get_error_result_from_sql(f'{RegressionPath}/{error_sql}')
            display(f"Testcase {test_id} Failed: {error}")
            save_output(test_id, description, "FAIL", error)
    else:
        display(f"Testcase {test_id} Failed due to post-test SQL")
        save_output(test_id, description, "FAIL", "Post-test SQL failed")
except Exception as e:
    display(f"Testcase {test_id} Failed due to exception: {e}")
    error = str(e)
    save_output(test_id, description, "FAIL", error)

In [0]:
test_case = test_cases_dict.get("TC_PUSH_024")
if test_case:
    test_id = test_case.get('TestID')
    test_case_desc = test_case.get('Test_Case_Description') 
    is_valid = FAM_emails_df.select("is_valid").first()["is_valid"]
    if is_valid:
        display("Testcase TC_PUSH_024 Passed")
        save_output("TC_PUSH_024", test_case_desc,"PASS")
    else:
        display("Testcase TC_PUSH_024 Failed")
        save_output("TC_PUSH_024", test_case_desc,"FAIL","FAM Mail not triggered")

In [0]:
test_case = test_cases_dict.get("TC_PUSH_025")
if test_case:
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    test_case_desc = test_case.get('Test_Case_Description')
    display(test_case_desc)
    Feed_complete_template = {
        "keywords": ["feed completion status", "complete","is_missing","missed_files","expectation_date","feed_details","arrival_date","extraction_type","centura_daily_autotestv6"]
    }
    is_valid_row = complete_feedname_df.select("is_valid").first()
    if is_valid_row:
        is_valid = is_valid_row["is_valid"]
        display("Testcase TC_PUSH_025 Passed")
        save_output("TC_PUSH_025", test_case_desc, "PASS")
    else:
        display("Testcase TC_PUSH_025 Failed")
        save_output("TC_PUSH_025", test_case_desc, "FAIL", "Feed Completion Mail not triggered")

In [0]:
test_case = test_cases_dict.get("TC_PUSH_026")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    test_case_desc = test_case.get('Test_Case_Description')  
    error_sql = test_case.get("ErrorSQL")
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_026:", sql_result)
    try:
        if sql_result == "PASS":
            file_path = f'{RegressionPath}/tc26/input_files/{test_file_names}'

            # Read source file with multiple extension support and count rows
            source_row_count = read_source_row_count(file_path) * 11 # multiplier using to make 1GB file
            display("source_row_count: ", source_row_count)

            # Read parquet file and count rows
            parquet_row_count = read_parquet_count_rows(Target_PB_Folder)
            display("parquet_row_count: ", parquet_row_count) 

        if sql_result == "PASS" and source_row_count == parquet_row_count:
            display("Testcase TC_PUSH_026 Passed")
            save_output("TC_PUSH_026", test_case_desc,"PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_026 Failed")
            save_output("TC_PUSH_026", test_case_desc,"FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))
        

In [0]:
test_case = test_cases_dict.get("TC_PUSH_027")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    test_case_desc = test_case.get('Test_Case_Description')  
    error_sql = test_case.get("ErrorSQL")

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_027:", sql_result)
    try:
        if sql_result == "PASS":
            file_path = f'{RegressionPath}/tc27/input_files/{test_file_names}'

            # Read source file with multiple extension support and count rows
            source_row_count = read_source_row_count(file_path)
            display("source_row_count: ", source_row_count)

            # Read folder and count parquet files
            parquet_file_count = read_parquet_file_count(Target_PB_Folder)
            display("parquet_file_count: ", parquet_file_count)

            # Read parquet file and count rows
            parquet_row_count = read_parquet_count_rows(Target_PB_Folder)
            display("parquet_row_count: ", parquet_row_count) 

        if sql_result == "PASS" and parquet_file_count == 1 and source_row_count == parquet_row_count:
            display("Testcase TC_PUSH_027 Passed")
            save_output("TC_PUSH_027",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_027 Failed")
            save_output("TC_PUSH_027",test_case_desc, "FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
test_case = test_cases_dict.get("TC_PUSH_028")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    test_case_desc = test_case.get('Test_Case_Description') 
    error_sql = test_case.get("ErrorSQL") 

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_028:", sql_result)
    try:
        if sql_result == "PASS":
            display("Testcase TC_PUSH_028 Passed")
            save_output("TC_PUSH_028",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_028 Failed")
            save_output("TC_PUSH_028",test_case_desc, "FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))


In [0]:
test_case = test_cases_dict.get("TC_PUSH_031")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    test_case_desc = test_case.get('Test_Case_Description') 
    error_sql = test_case.get("ErrorSQL") 
    source_path = test_case.get('source_file')
    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_031:", sql_result)
    test_case = test_cases_dict.get('TC_PUSH_031')
    try:
        source_row_count = read_source_row_count(RegressionPath+source_path)
        processed_row_count = read_parquet_count_rows(Target_PB_Folder)
        display(f'source row count : {source_row_count}')
        display(f'processed row count : {processed_row_count}')
        if sql_result == "PASS" and source_row_count == processed_row_count:
            display("Testcase TC_PUSH_031 Passed")
            save_output("TC_PUSH_031",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_031 Failed")
            save_output("TC_PUSH_031",test_case_desc, "FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))


In [0]:
test_case = test_cases_dict.get("TC_PUSH_029")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    test_case_desc = test_case.get('Test_Case_Description')  
    error_sql = test_case.get("ErrorSQL")

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_029:", sql_result)
    try:
        if sql_result == "PASS":
            file_path = f'{RegressionPath}/tc29/input_files/{test_file_names}'

            # Read source file with multiple extension support and count rows
            source_row_count = read_source_row_count(file_path) * 11 # multiplier using to make 1GB file
            display("source_row_count: ", source_row_count)

            # Read parquet file and count rows
            parquet_row_count = read_parquet_count_rows(Target_PB_Folder)
            display("parquet_row_count: ", parquet_row_count) 

            listOfFiles = dbutils.fs.ls(Target_PB_Folder)
            for file in listOfFiles:
                fileSize = file.size
                display(f"File: {file.name}, Size: {fileSize} bytes")
                if fileSize < 512000000:
                    sizeValidation = "PASS"
                else:
                    sizeValidation = "FAIL"      
            display("Parquet file count in folder:", len(listOfFiles))
            display("sizeValidation:", sizeValidation)

        if sql_result == "PASS" and source_row_count == parquet_row_count and sizeValidation == "PASS":
            display("Testcase TC_PUSH_029 Passed")
            save_output("TC_PUSH_029",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_029 Failed")
            save_output("TC_PUSH_029",test_case_desc, "FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
test_case = test_cases_dict.get("TC_PUSH_030")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    test_case_desc = test_case.get('Test_Case_Description')  
    error_sql = test_case.get("ErrorSQL")

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_030:", sql_result)
    try:
        if sql_result == "PASS":
            file_path = f'{RegressionPath}/tc30/input_files/{test_file_names}'

            # Read source file with multiple extension support and count rows
            source_row_count = read_source_row_count(file_path)
            display("source_row_count: ", source_row_count)

            # Read folder and count parquet files
            parquet_file_count = read_parquet_file_count(Target_PB_Folder)
            display("parquet_file_count: ", parquet_file_count)

            # Read parquet file and count rows
            parquet_row_count = read_parquet_count_rows(Target_PB_Folder)
            display("parquet_row_count: ", parquet_row_count)

        if sql_result == "PASS" and parquet_file_count == 1 and source_row_count == parquet_row_count:
            display("Testcase TC_PUSH_030 Passed")
            save_output("TC_PUSH_030", test_case_desc,"PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_030 Failed")
            save_output("TC_PUSH_030", test_case_desc,"FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
test_case = test_cases_dict.get("TC_PUSH_032")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    test_case_desc = test_case.get('Test_Case_Description') 
    error_sql = test_case.get("ErrorSQL") 

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_032:", sql_result)
    try:
        if sql_result == "PASS":
            display("Testcase TC_PUSH_032 Passed")
            save_output("TC_PUSH_032",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_032 Failed")
            save_output("TC_PUSH_032",test_case_desc, "FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))


In [0]:
test_case = test_cases_dict.get("TC_PUSH_033")
if test_case:
    test_id = test_case.get('TestID')
    test_case_desc = test_case.get('Test_Case_Description') 
    display(test_case_desc)
    is_valid_row = incomplete_feedname_df.select("is_valid").first()
    if is_valid_row is not None and is_valid_row["is_valid"]:
        display("Testcase TC_PUSH_033 Passed")
        save_output("TC_PUSH_033", test_case_desc,"PASS")
    else:
        display("Testcase TC_PUSH_033 Failed")
        save_output("TC_PUSH_033", test_case_desc,"FAIL","Complete feed Mail not triggered")

In [0]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.datafactory import DataFactoryManagementClient
from azure.mgmt.datafactory.models import RunFilterParameters
from datetime import datetime, timedelta

test_case = test_cases_dict.get("TC_PUSH_034")
test_id = test_case.get('TestID')
test_file_names = test_case.get('Test_File_Names')
test_case_desc = test_case.get('Test_Case_Description')
feed_name = test_case.get('Feed_Name')

try:
    DVP_Validation = False
    for run in DVP_runs.value:
        if  run.parameters['FeedName'] == feed_name  and test_file_names in str(run) and run.status == 'Succeeded':
            Validation = True
            break
    if Validation == True:
        display("Testcase TC_PUSH_034 Passed")
        save_output("TC_PUSH_034", test_case_desc,"PASS")
    else:
        display("Testcase TC_PUSH_034 Failed")
        save_output("TC_PUSH_034", test_case_desc,"FAIL","DVP trigger failed ")
except Exception as e:
    display(f"Testcase {test_id} Failed due to exception: {e}")
    save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
test_case = test_cases_dict.get("TC_PUSH_036")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_case_desc = test_case.get('Test_Case_Description') 
    display(test_case_desc)
    is_valid_row = incomplete_feedname_df.select("is_valid").first()
    if is_valid_row is not None and is_valid_row["is_valid"]:
        display("Testcase TC_PUSH_036 Passed")
        save_output("TC_PUSH_036", test_case_desc,"PASS")
    else:
        display("Testcase TC_PUSH_036 Failed")
        save_output("TC_PUSH_036", test_case_desc,"FAIL","Incomplete feed Mail not triggered")

In [0]:
# TC_PUSH_037: Trigger FAM Function App and validate response
test_case_37 = test_cases_dict.get("TC_PUSH_037")
if test_case_37:
    test_case_desc_37 = test_case_37.get('Test_Case_Description')
    test_id_37 = test_case_37.get('TestID')
    display(f"{test_id_37} : {test_case_desc_37}")
    master_key = get_secret(constants.Function_App_MasterKey_Secret_Name, key_vault_name)
    FAM_response = trigger_azure_function(constants.function_app_name, constants.FAM_Function_name, master_key)
    if FAM_response.get("status_code") == 202:
        df = fetch_today_logic_app_runs(constants.logic_app_subscription_id, constants.logic_app_resource_group, constants.logic_app_name)
        Updated_FAM_emails_count = df.filter(validate_keywords_udf(FAM_keywords)(col("response"))).count()
        if Updated_FAM_emails_count > FAM_email_count:
            display("TC_PUSH_037 Passed")
            save_output("TC_PUSH_037", test_case_desc_37, "PASS", "received the FAM mail when triggered manually")           
        else:
            display("TC_PUSH_037 Failed")
            save_output("TC_PUSH_037", test_case_desc_37, "FAIL", f"Not Received the mail when triggered manually ")
    else:
        display("TC_PUSH_037 Failed")
        save_output("TC_PUSH_037", test_case_desc_37, "FAIL", f"Not able to trigger the Function App {constants.FAM_Function_name}")

In [0]:
test_case = test_cases_dict.get("TC_PUSH_038")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    test_case_desc = test_case.get('Test_Case_Description')  
    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    error_sql = test_case.get("ErrorSQL")
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_038:", sql_result)
    try:
        if sql_result == "PASS":
            file_path = f'{RegressionPath}/tc38/input_files/{test_file_names}'
            source_row_count = read_source_row_count(file_path)
            display("source_row_count: ", source_row_count)
            parquet_row_count = read_parquet_count_rows(Target_PB_Folder)
            display("parquet_row_count: ", parquet_row_count)
        if sql_result == "PASS" and source_row_count == parquet_row_count:
            display("Testcase TC_PUSH_038 Passed")
            save_output("TC_PUSH_038",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_038 Failed")
            save_output("TC_PUSH_038", test_case_desc,"FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))
        

In [0]:
test_case = test_cases_dict.get("TC_PUSH_039")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    test_case_desc = test_case.get('Test_Case_Description')  
    error_sql = test_case.get("ErrorSQL")
    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_039:", sql_result)
    try:
        if sql_result == "PASS":
            display("Testcase TC_PUSH_039 Passed")
            save_output("TC_PUSH_039",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_039 Failed")
            save_output("TC_PUSH_039",test_case_desc, "FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))
        

In [0]:
test_case = test_cases_dict.get("TC_PUSH_040")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    test_case_desc = test_case.get('Test_Case_Description')  
    error_sql = test_case.get("ErrorSQL")

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_040:", sql_result)
    try:
        if sql_result == "PASS":
            file_path = f'{RegressionPath}/tc40/input_files/{test_file_names}'

            # Read source file with multiple extension support and count rows
            source_row_count = read_source_row_count(file_path)
            display("source_row_count: ", source_row_count)

            # Read folder and count parquet files
            parquet_file_count = read_parquet_file_count(Target_PB_Folder)
            display("parquet_file_count: ", parquet_file_count)

            # Read parquet file and count rows
            parquet_row_count = read_parquet_count_rows(Target_PB_Folder)
            display("parquet_row_count: ", parquet_row_count)

        if sql_result == "PASS" and parquet_file_count == 1 and source_row_count == parquet_row_count:
            display("Testcase TC_PUSH_040 Passed")
            save_output("TC_PUSH_040", test_case_desc,"PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_040 Failed")
            save_output("TC_PUSH_040",test_case_desc, "FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
test_case = test_cases_dict.get("TC_PUSH_041")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    test_case_desc = test_case.get('Test_Case_Description')  
    error_sql = test_case.get("ErrorSQL")
    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_041:", sql_result)
    try:
        if sql_result == "PASS":
            file_path = f'{RegressionPath}/tc41/input_files/{test_file_names}'

            # Read source file with multiple extension support and count rows
            source_row_count = read_source_row_count(file_path)
            display("source_row_count: ", source_row_count)

            # Read folder and count parquet files
            parquet_file_count = read_parquet_file_count(Target_PB_Folder)
            display("parquet_file_count: ", parquet_file_count)

            # Read parquet file and count rows
            parquet_row_count = read_parquet_count_rows(Target_PB_Folder)
            display("parquet_row_count: ", parquet_row_count)

        if sql_result == "PASS" and parquet_file_count == 1 and source_row_count == parquet_row_count:
            display("Testcase TC_PUSH_041 Passed")
            save_output("TC_PUSH_041",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_041 Failed")
            save_output("TC_PUSH_041",test_case_desc, "FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))
        

In [0]:
test_case = test_cases_dict.get("TC_PUSH_042")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    test_case_desc = test_case.get('Test_Case_Description') 
    error_sql = test_case.get("ErrorSQL") 

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_042:", sql_result)
    try:
        if sql_result == "PASS":
            file_path = f'{RegressionPath}/tc42/input_files/{test_file_names}'

            # Read source file with multiple extension support and count rows
            source_row_count = read_source_row_count(file_path)
            display("source_row_count: ", source_row_count)

            # Read folder and count parquet files
            parquet_file_count = read_parquet_file_count(Target_PB_Folder)
            display("parquet_file_count: ", parquet_file_count)

            # Read parquet file and count rows
            parquet_row_count = read_parquet_count_rows(Target_PB_Folder)
            display("parquet_row_count: ", parquet_row_count)

        if sql_result == "PASS" and parquet_file_count == 1 and source_row_count == parquet_row_count:
            display("Testcase TC_PUSH_042 Passed")
            save_output("TC_PUSH_042", test_case_desc,"PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_042 Failed")
            save_output("TC_PUSH_042", test_case_desc,"FAIL",error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
test_case = test_cases_dict.get("TC_PUSH_043")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    test_case_desc = test_case.get('Test_Case_Description')  
    display(test_case_desc)
    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    rerun_count_sql = test_case.get('PreTest_SQL')
    Archive_path = test_case.get('Archive_Folder')
    error_sql = test_case.get("ErrorSQL")

    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_043:", sql_result)
    try:
        if sql_result == "PASS":
            file_path = f'{RegressionPath}/tc43/input_files/{test_file_names}'

            # Read source file with multiple extension support and count rows
            source_row_count = read_source_row_count(file_path)
            display("source_row_count: ", source_row_count)

            # Read folder and count parquet files
            parquet_file_count = read_parquet_file_count(Target_PB_Folder)
            display("parquet_file_count: ", parquet_file_count)

            # Read parquet file and count rows
            parquet_row_count = read_parquet_count_rows(Target_PB_Folder)
            display("parquet_row_count: ", parquet_row_count)

            # Get rerun count from database
            re_run_count_df = get_rerun_count(rerun_count_sql)
            re_run_count = re_run_count_df.iloc[0]['re_run_count']
            display("re_run_count_df: ", re_run_count)

            # Read archive folder using rerun count and count parquet files in archive folder 
            archive_file_count = read_parquet_file_count(Archive_path.format(re_run_count = re_run_count,env=env))
            display("archive_file_count: ", archive_file_count)

        if sql_result == "PASS" and parquet_file_count == 1 and source_row_count == parquet_row_count and archive_file_count == 1:
            display("Testcase TC_PUSH_043 Passed")
            save_output("TC_PUSH_043",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_043 Failed")
            save_output("TC_PUSH_043",test_case_desc, "FAIL",error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
test_case = test_cases_dict.get("TC_PUSH_044")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    test_case_desc = test_case.get('Test_Case_Description')  
    error_sql = test_case.get("ErrorSQL")
    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_044:", sql_result)
    try:
        if sql_result == "PASS":
            file_path = f'{RegressionPath}/tc44/input_files/{test_file_names}'

            # Read source file with multiple extension support and count rows
            source_row_count = read_source_row_count(file_path)
            display("source_row_count: ", source_row_count)

            # Read folder and count parquet files
            parquet_file_count = read_parquet_file_count(Target_PB_Folder)
            display("parquet_file_count: ", parquet_file_count)

            # Read parquet file and count rows
            parquet_row_count = read_parquet_count_rows(Target_PB_Folder)
            display("parquet_row_count: ", parquet_row_count)

            # Archive folder check, Extracted_holding folder check,Auto Feed completion status mail check--Pending

        if sql_result == "PASS" and parquet_file_count == 1 and source_row_count == parquet_row_count:
            display("Testcase TC_PUSH_044 Passed")
            save_output("TC_PUSH_044",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_044 Failed")
            save_output("TC_PUSH_044",test_case_desc, "FAIL",error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.datafactory import DataFactoryManagementClient
from azure.mgmt.datafactory.models import RunFilterParameters
from datetime import datetime, timedelta

test_case = test_cases_dict.get("TC_PUSH_045")
post_test_sql = test_case.get('PostTest_SQL')
test_id = test_case.get('TestID')
test_file_names = test_case.get('Test_File_Names')
test_case_desc = test_case.get('Test_Case_Description') 
error_sql = test_case.get("ErrorSQL")

try:
    cnt = 0
    for run in DAP_Orchestration_runs.value:
        if run.parameters['FeedName']  == 'centura_daily_autotest':
            cnt+=1
    if cnt > 20:
        display("Testcase TC_PUSH_045 Passed")
        save_output("TC_PUSH_045", test_case_desc,"PASS")
    else:
        display("Testcase TC_PUSH_045 Failed")
        save_output("TC_PUSH_045", test_case_desc,"FAIL",f"{cnt} pipelines triggered in last 60 minutes")
except Exception as e:
    display(f"Testcase {test_id} Failed due to exception: {e}")
    save_output(test_id, test_case_desc, "FAIL", str(e))
    

In [0]:
test_case = test_cases_dict.get("TC_PUSH_046")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    test_case_desc = test_case.get('Test_Case_Description')  
    error_sql = test_case.get("ErrorSQL")
    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    display(sql_file_path)
    sql_result = run_post_test_sql(sql_file_path,test_id)
    validator = True
    sources = test_case.get("Sources")
    display("Query execution status of TC_PUSH_046:", sql_result)
    try:
        if sql_result == "PASS":
            targetPaths = test_case.get('Target_PB_Folder')
            sourcePaths = test_case.get('Source_Path')
            results = {}
            for source in sources:
                results[source] = {}
                parts_count = 3 if source == "claims" else 2
                for i in range(1, parts_count + 1):
                    results[source][f"part{i}"] = {}
                    tgt_directory = targetPaths[source].format(env=env) + str(i)
                    results[source][f"part{i}"]["target_count"] = read_parquet_count_rows(tgt_directory)
                    src_directory = sourcePaths[source].format(env=env , i=i)
                    results[source][f"part{i}"]["source_count"] = read_source_row_count(RegressionPath+src_directory)
            display(results)
            for source, parts in results.items():
                for key, value in parts.items():
                    if value["target_count"] != value["source_count"]:
                        validator = False
        if sql_result == "PASS" and validator:
            display("Testcase TC_PUSH_046 Passed")
            save_output("TC_PUSH_046", test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_046 Failed")
            save_output("TC_PUSH_046", test_case_desc, "FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
test_case = test_cases_dict.get("TC_PUSH_047")
post_test_sql = test_case.get('PostTest_SQL')
test_id = test_case.get('TestID')
test_file_names = test_case.get('Test_File_Names')
Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
test_case_desc = test_case.get('Test_Case_Description')
error_sql = test_case.get("ErrorSQL")

sql_file_path = f'{RegressionPath}/{post_test_sql}'
display(test_case_desc)
# Call run_sql_file() function to get test result
sql_result = run_post_test_sql(sql_file_path,test_id)
try:
    if sql_result == "PASS":
        display("Testcase TC_PUSH_047 Passed")
        save_output("TC_PUSH_047", test_case_desc,"PASS")
    else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_047 Failed")
            save_output("TC_PUSH_047",test_case_desc, "FAIL",error)
except Exception as e:
    display(f"Testcase {test_id} Failed due to exception: {e}")
    save_output(test_id, test_case_desc, "FAIL", str(e))
    

In [0]:

test_case = test_cases_dict.get("TC_PUSH_048")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    test_case_desc = test_case.get('Test_Case_Description')
    error_sql = test_case.get("ErrorSQL")
sql_file_path = f'{RegressionPath}/{post_test_sql}'
display(test_case_desc)
# Call run_sql_file() function to get test result
sql_result = run_post_test_sql(sql_file_path,test_id)
parts = [2,3]
targetPath = test_case.get('Target_PB_Folder')
try : 
    for part in parts : 
        tgt_directory = targetPath.format(part=part,env=env)
except:
    display("Testcase TC_PUSH_048 Failed")
    save_output("TC_PUSH_048",test_case_desc, "FAIL","the expecting part files are not there in the landed path") 
if sql_result == "PASS":
    display("Testcase TC_PUSH_048 Passed")
    save_output("TC_PUSH_048", test_case_desc,"PASS")
else:
        sql_error_file_path = f'{RegressionPath}/{error_sql}'
        error=get_error_result_from_sql(sql_error_file_path)
        display("Testcase TC_PUSH_048 Failed")
        save_output("TC_PUSH_048",test_case_desc, "FAIL",error)   
    



In [0]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.datafactory import DataFactoryManagementClient
from azure.mgmt.datafactory.models import RunFilterParameters
from datetime import datetime, timedelta
import json

test_case = test_cases_dict.get("TC_PUSH_049")
if test_case:
    test_id = test_case.get('TestID')
    file_names_to_check = test_case.get('Test_File_Names')
    test_case_desc = test_case.get('Test_Case_Description')  
    present_files = []
    try:
        for run in FAV_runs.value:
            if run.status == "Succeeded":
                for file in file_names_to_check:
                    if file  in str(run):
                        present_files.append(file)
        for file in present_files:    display(file)
        if all(file in present_files for file in file_names_to_check):
            display("All files are present.")
            display("TC_PUSH_049 Passed")
            save_output("TC_PUSH_049",test_case_desc, "PASS")        
        else:
            missing_files = [file for file in file_names_to_check if file not in present_files]
            display(f"Missing files: {missing_files}")
            save_output("TC_PUSH_049",test_case_desc, "FAIL",f"fav trigger failed for the files : {missing_files}")
    except Exception as e:
        display(f"Error: {e}")
        save_output("TC_PUSH_049",test_case_desc, "FAIL",str(e))

In [0]:
import logging
from azure.monitor.query import LogsQueryClient, LogsQueryStatus
from azure.identity import DefaultAzureCredential
from datetime import timedelta
import re

# Configuration
CONFIG = {
    "app_insights_app_id": "9031317c-052e-4f0d-a7a6-a8c73b3049ba",
    "timespan_hours": 2
}
error = None

def get_ticket_numbers(app_insights_app_id, timespan):
    try:
        credential = DefaultAzureCredential()
        client = LogsQueryClient(credential)

        query = """
        AppTraces
        | where Message contains "ticket"
        | project TimeGenerated, Message
        """

        response = client.query_workspace(
            workspace_id=app_insights_app_id,
            query=query,
            timespan=timespan
        )

        if response.status == LogsQueryStatus.SUCCESS:
            ticket_numbers = []
            for table in response.tables:
                for row in table.rows:
                    message = row[1]  # Access the 'Message' field
                    match = re.search(r"\bUS\d+\b", message)
                    if match:
                        ticket_numbers.append(match.group(0))
            return ticket_numbers
        else:
            error = f"Application Insights query failed: {response.status}"
            return None
    except Exception as e:
        error = f"An unexpected error occurred: {e}"
        return None


if __name__ == "__main__":
    test_case = test_cases_dict.get("TC_PUSH_050")
    test_id = test_case.get('TestID')
    file_names_to_check = test_case.get('Test_File_Names')
    test_case_desc = test_case.get('Test_Case_Description') 
    timespan = timedelta(hours=CONFIG["timespan_hours"])
    ticket_numbers = get_ticket_numbers(CONFIG["app_insights_app_id"], timespan)
    if ticket_numbers:
        display("Ticket Numbers Found:")
        for num in ticket_numbers:
            display(num)
        save_output("TC_PUSH_050",test_case_desc, "PASS", ",".join(ticket_numbers))

    else:
        error = "No ticket numbers found or an error occurred."
        display('no rally ticket created')
        save_output("TC_PUSH_050",test_case_desc, "FAIL",error)


In [0]:
test_case = test_cases_dict.get("TC_PUSH_051")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    error_sql = test_case.get("ErrorSQL")
    test_case_desc = test_case.get('Test_Case_Description')
    field_to_check = test_case.get('Field_to_check')
    source_path = test_case.get('Source_path')

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_051:", sql_result)
    try:
        if sql_result == "PASS":
          df = spark.read.parquet(Target_PB_Folder)
          null_count = df.select(field_to_check).filter(df[field_to_check].isNull()).count()
          processed_row_count = read_parquet_count_rows(Target_PB_Folder)
          source_row_count = read_source_row_count(RegressionPath+source_path)
        if sql_result == "PASS" and null_count == 3 and processed_row_count == source_row_count:
            display("Testcase TC_PUSH_051 Passed")
            save_output("TC_PUSH_051",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_051 Failed")
            save_output("TC_PUSH_051", test_case_desc,"FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.jobs import RunLifeCycleState, RunResultState
from datetime import datetime, timedelta


test_case = test_cases_dict.get("TC_PUSH_052")
if test_case:
    
    workspace_url = "https://adb-4795873923915319.19.azuredatabricks.net"
    personal_access_token = get_secret(constants.PAT_Secret_name, key_vault_name)
    try:
        w = WorkspaceClient(
            host=workspace_url,
            token=personal_access_token
        )
        display(f"Successfully connected to workspace: {w.config.host}")
    except Exception as e:
        display(f"Error initializing WorkspaceClient: {e}")
        display("Ensure you are running this notebook in a Databricks workspace and the SDK is installed.")

    # You can adjust these filters. If all are None, it gets all runs (could be many).
    start_time_from_ms = int((datetime.utcnow() - timedelta(minutes=120)).timestamp() * 1000)
    start_time_to_ms = None # No upper bound for start time, effectively "up to now"
    completed_only_filter = True # Set to True to only get completed runs, False for active and completed
    limit_per_page = 25 # Max allowed by API per page is 25 for list_runs
    max_runs_to_fetch = 400 # To prevent fetching too many runs in this example

    display(f"Fetching runs with parameters:")
    display(f"  Job ID: {'All'}")
    display(f"  Start Time From (ms epoch): {start_time_from_ms}")
    display(f"  Start Time To (ms epoch): {start_time_to_ms}")
    display(f"  Completed Only: {completed_only_filter}")
    display(f"  Max runs to fetch: {max_runs_to_fetch}")
    display("-" * 30)
    ADB_validation = False
    try:
        for run in w.jobs.list_runs(
            completed_only=completed_only_filter,
            start_time_from=start_time_from_ms,
            start_time_to=start_time_to_ms,
            expand_tasks=False # Set to True if you need task details, makes response larger
        ):
            if "preprocessing" in str(run.run_name):
                ADB_validation = True
    except Exception as e:
        display(f"Error fetching job runs: {e}")
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    error_sql = test_case.get("ErrorSQL")
    test_case_desc = test_case.get('Test_Case_Description')
    source_path = test_case.get('Source_path')

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_052:", sql_result)
    display(ADB_validation)
    try:
        if sql_result == "PASS"   and ADB_validation:
            display("Testcase TC_PUSH_052 Passed")
            save_output("TC_PUSH_052",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_052 Failed")
            save_output("TC_PUSH_052", test_case_desc,"FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
test_case = test_cases_dict.get("TC_PUSH_053")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder')
    source_folder = test_case.get('Source_Folder')
    test_case_desc = test_case.get('Test_Case_Description')  
    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    error_sql = test_case.get("ErrorSQL")
    display(test_case_desc)
    file_path = test_case.get("sorce_folder")

    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path, test_id)

    display("Query execution status of TC_PUSH_053:", sql_result)
    if sql_result == "PASS":    
        try:
            source_counts = {}
            i = 1
            for file_name in test_file_names:
                source_counts[i] = read_source_row_count(file_path=RegressionPath+file_path.format(  file_name = file_name))
                i += 1
            target_counts = {}
            for part in range(1,3):
                target_counts[part] = read_parquet_count_rows(file_path=Target_PB_Folder.format(env = env ,part=part)) 

        except Exception as e:
            display(f"An error occurred: {e}")
        if sql_result == "PASS" and source_counts == target_counts :
            display("Testcase TC_PUSH_053 Passed")
            save_output("TC_PUSH_053", test_case_desc,"PASS")
    else:
        sql_error_file_path = f'{RegressionPath}/{error_sql}'
        res=get_error_result_from_sql(sql_error_file_path)
        display("Testcase TC_PUSH_053 Failed")
        save_output("TC_PUSH_053", test_case_desc,"FAIL", error)

In [0]:
test_case = test_cases_dict.get("TC_PUSH_054")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    test_case_desc = test_case.get('Test_Case_Description') 
    error_sql = test_case.get("ErrorSQL") 
    file_name = test_case.get('Test_File_Names')
    source_path = test_case.get('source_path')

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_054:", sql_result)
    try:
        source_row_count = spark.read.json('file:'+RegressionPath + source_path).count()
        processed_row_count = read_parquet_count_rows(Target_PB_Folder)
        display(f'source row count : {source_row_count}'  )
        display(f'processed row count : {processed_row_count}'  )
        if sql_result == "PASS" and processed_row_count == source_row_count:

            display("Testcase TC_PUSH_054 Passed")
            save_output("TC_PUSH_054",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_054 Failed")
            save_output("TC_PUSH_054",test_case_desc, "FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))


In [0]:
test_case = test_cases_dict.get("TC_PUSH_055")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    test_case_desc = test_case.get('Test_Case_Description') 
    error_sql = test_case.get("ErrorSQL") 

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_055:", sql_result)
    try:
        if sql_result == "PASS":
            display("Testcase TC_PUSH_055 Passed")
            save_output("TC_PUSH_055",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_055 Failed")
            save_output("TC_PUSH_055",test_case_desc, "FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))


In [0]:
test_case = test_cases_dict.get("TC_PUSH_056")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    test_case_desc = test_case.get('Test_Case_Description') 
    error_sql = test_case.get("ErrorSQL") 
    source_path = test_case.get('Source_path')
    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_056:", sql_result)
    try:
        source_row_count = read_source_row_count(RegressionPath+source_path)
        processed_row_count = read_parquet_count_rows(Target_PB_Folder)
        if sql_result == "PASS" and source_row_count == processed_row_count:
            display("Testcase TC_PUSH_056 Passed")
            save_output("TC_PUSH_056",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_056 Failed")
            save_output("TC_PUSH_056",test_case_desc, "FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
from pyspark.sql.functions import col, lower
from pyspark.sql import DataFrame
test_case = test_cases_dict.get("TC_PUSH_057")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    test_case_desc = test_case.get('Test_Case_Description') 
    error_sql = test_case.get("ErrorSQL") 
    source_path = test_case.get('Source_Path')

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_057:", sql_result)
    
    try:
        src_df = spark.read.csv(f'file:{RegressionPath}{source_path}', header=True)
        tgt_df = spark.read.parquet(Target_PB_Folder)
        src_null_count = src_df.filter(lower(col("claim_id")) == "null").count()
        tgt_null_count = tgt_df.filter(col("claim_id").isNull()).count()
        if sql_result == "PASS" and src_null_count == tgt_null_count:
            display("Testcase TC_PUSH_057 Passed")
            save_output("TC_PUSH_057",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_057 Failed")
            save_output("TC_PUSH_057",test_case_desc, "FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
from pyspark.sql.functions import trim
test_case = test_cases_dict.get("TC_PUSH_058")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_file_names = test_case.get('Test_File_Names')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    test_case_desc = test_case.get('Test_Case_Description') 
    error_sql = test_case.get("ErrorSQL") 
    source_path = test_case.get('Source_Path')

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_057:", sql_result)
    
    try:
        src_df = spark.read.csv(f'file:{RegressionPath}{source_path}', header=True)
        tgt_df = spark.read.parquet(Target_PB_Folder)
        src_patient_names = src_df.select("patient_name").distinct()
        tgt_patient_names = tgt_df.select("patient_name").distinct()
        trimmed_src_patient_names = src_df.select(trim("patient_name")).distinct()
        non_trim_patient_name_match = src_patient_names.exceptAll(tgt_patient_names).count() == 0 and tgt_patient_names.exceptAll(src_patient_names).count() == 0
        patient_name_match = trimmed_src_patient_names.exceptAll(tgt_patient_names).count() == 0 and tgt_patient_names.exceptAll(trimmed_src_patient_names).count() == 0

        if sql_result == "PASS" and non_trim_patient_name_match == False and patient_name_match == True:
            display(f"Testcase {test_id} Passed")
            save_output(test_id,test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display(f"Testcase {test_id} Failed")
            save_output("TC_PUSH_057",test_case_desc, "FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
test_case = test_cases_dict.get("TC_PUSH_059")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_case_desc = test_case.get('Test_Case_Description')
    ADF_status_template = {
    "keywords": ["This is to notify you about a failure in the ADF process during the extraction of data", "ADF pipeline run ID","ADF Process Failure Notification","ADF Run ID","please refer to the Error Handling in ADF","Feed Name","Source File Name","Processed File Name","Error Code","Error Message"]
    }
    is_valid = adf_failure_df.select("is_valid").first()["is_valid"]
    if is_valid:
        display(f"Testcase {test_id} Passed")
        save_output(test_id,test_case_desc, "PASS")
    else:
        display(f"Testcase {test_id} Failed")
        save_output(test_id,test_case_desc, "FAIL","Mail not triggered")


     

In [0]:
test_case = test_cases_dict.get("TC_PUSH_060")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    Target_PB_Folder = test_case.get('Target_PB_Folder').format(env=env)
    test_case_desc = test_case.get('Test_Case_Description') 
    error_sql = test_case.get("ErrorSQL") 
    source_path = test_case.get('Source_Path')

    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_060:", sql_result)
    try:
        source_row_count = read_source_row_count(f'{RegressionPath}{source_path}')
        processed_row_count = read_parquet_count_rows(Target_PB_Folder)
        if sql_result == "PASS" and source_row_count == processed_row_count:
            display("Testcase TC_PUSH_060 Passed")
            save_output("TC_PUSH_060",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_060 Failed")
            save_output("TC_PUSH_060",test_case_desc, "FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
test_case = test_cases_dict.get("TC_PUSH_061")
if test_case:
    post_test_sql = test_case.get('PostTest_SQL')
    test_id = test_case.get('TestID')
    test_case_desc = test_case.get('Test_Case_Description') 
    error_sql = test_case.get("ErrorSQL") 
    sql_file_path = f'{RegressionPath}/{post_test_sql}'
    display(test_case_desc)
    # Call run_sql_file() function to get test result
    sql_result = run_post_test_sql(sql_file_path,test_id)
    display("Query execution status of TC_PUSH_061:", sql_result)
    try:
        if sql_result == "PASS" :
            display("Testcase TC_PUSH_061 Passed")
            save_output("TC_PUSH_061",test_case_desc, "PASS")
        else:
            sql_error_file_path = f'{RegressionPath}/{error_sql}'
            error=get_error_result_from_sql(sql_error_file_path)
            display("Testcase TC_PUSH_061 Failed")
            save_output("TC_PUSH_061",test_case_desc, "FAIL", error)
    except Exception as e:
        display(f"Testcase {test_id} Failed due to exception: {e}")
        save_output(test_id, test_case_desc, "FAIL", str(e))

In [0]:
import csv  
from datetime import datetime  
  
# Define the current timestamp  
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')  
  
# Define the path to save the CSV report with timestamp suffix  
csv_report_path = f'/dbfs/mnt/data/{env}/Test_files/regression/test_case_report/test_case_report_{timestamp}.csv'  

with open(csv_report_path, mode='w', newline='') as file:  
    writer = csv.writer(file)  
    writer.writerow(['Test Case',  'Status', 'Description' , 'Comments'])  
    for test_case, status in outputs.items():  
        writer.writerow([test_case, status['result'], status['description'], status['error']])  
  
# Display the CSV file path  
df = spark.read.csv(f'dbfs:/mnt/data/{env}/Test_files/regression/test_case_report/test_case_report_{timestamp}.csv', header=True) 
ordered_df = df.orderBy("Test Case") 
display(ordered_df)

In [0]:
total_tests = df.count()
passed_tests = df.filter(df["Status"] == 'PASS').count()
failed_tests = df.filter(df["Status"] == 'FAIL').count()
pass_percentage = (passed_tests / total_tests) * 100
display(pass_percentage)

In [0]:
import pandas as pd
from datetime import datetime, timezone, timedelta
import requests

def create_email_body_from_csv_pandas(ordered_df):
    """Creates an email body with a CSV file as a formatted table using pandas."""
    try:
        df = ordered_df.toPandas()
        html_table = df.to_html(index=False, justify='center', border=1) #index=False removes row index,justify center aligns cells
        email_body = f"""<h2>Please find the below test case report :</h2>\n <p>Total Test Cases: {total_tests}</p>\n<p>Passed Test Cases: {passed_tests}</p>\n<p>Failed Test Cases: {failed_tests}</p>\n<p>Pass Percentage: {pass_percentage:.2f}%</p>\n\n{html_table}""" # Add HTML header
        return email_body
    except FileNotFoundError:
        display(f"Error: CSV file not found at {ordered_df}")
        return None
    except Exception as e:
        display(f"An error occurred: {e}")
        return None

body = create_email_body_from_csv_pandas(ordered_df)

dap_nonprod_emailer_URL = get_secret(dap_nonprod_emailer_URL_nm, key_vault_name)
today = datetime.now().astimezone().astimezone(timezone(timedelta(hours=5, minutes=30))).strftime("%Y-%m-%d %H:%M")
subject = f"DAP regression automation {env.upper()} Test Case Report - {today}"
recipients = jsonObject.get('Recipients')
sender = "odedap@omail.o360.cloud"
json_data = {"message": body,"subject": subject, "receiver":recipients, "from": sender} 

output = requests.post(dap_nonprod_emailer_URL, json=json_data)
display(output.status_code)



In [0]:


def get_run_details(run_id , DAP_Orchestration_runs):
    for run in DAP_Orchestration_runs.value:
        if run.run_id == run_id:
            return {
                "id" : run_id,
                "run_status": run.status,
                "message": run.message,
                "run_duration_min": f"{round((run.duration_in_ms or 0) / 60000, 2):.2f}"
            }
run_details = {}
def store_run_output(file_name, output):
    run_details[file_name] = output

df_runs = execute_query(ADF_run_from_feed_master_query)

def process_all_runs(df_runs,DAP_Orchestration_runs):
    for _,row in df_runs.iterrows():
        run_id = row['run_id']
        file_name = row['file_name']
        run_details = get_run_details(run_id,DAP_Orchestration_runs)
        store_run_output(file_name,run_details)
process_all_runs(df_runs,DAP_Orchestration_runs)

In [0]:
# Define the path to save the CSV report with timestamp suffix  
run_details_path = f'/dbfs/mnt/data/{env}/Test_files/regression/test_case_report/ADF_Run_Details_{timestamp}.csv' 
with open(run_details_path, mode='w', newline='') as file:  
    writer = csv.writer(file)  
    writer.writerow(['File_Name',  'run_id', 'run_status' , 'Error Message', 'run_duration_min'])  
    for file_name, status in run_details.items():  
        if status is not None:
            writer.writerow([
                file_name, 
                status.get('id', 'N/A'), 
                status.get('run_status', 'N/A'), 
                status.get('message', 'N/A'), 
                status.get('run_duration_min', 'N/A')
            ])
        else:
            writer.writerow([
                file_name, 
                'N/A', 
                'N/A', 
                'N/A', 
                'N/A'
            ])

In [0]:
# Processing test case and ADF run details, then saving them to Delta tables

from pyspark.sql.functions import current_timestamp, date_format, col, sha2, lit
from pyspark.sql.types import DoubleType

combined_output = {
    "test_case_report": outputs,
    "adf_run_details": run_details
}
run_timestamp = spark.sql("SELECT current_timestamp()").collect()[0][0]

test_case_records = []
test_report_dict = combined_output.get('test_case_report', {})

for test_case_name, details in test_report_dict.items():
    test_case_records.append({
        "Test Case": test_case_name,
        "Status": details.get('result','N/A'),
        "Description": details.get('description','N/A'),
        "Error Message": None if details.get('error') == 'null' else details.get('error'),
        "environment": env
    })

if test_case_records:
    test_case_df = spark.createDataFrame(test_case_records)
    
    for col_name in test_case_df.columns:
        new_col_name = col_name.replace(" ", "_").replace(";", "_").replace("{", "_") \
                               .replace("}", "_").replace("(", "_").replace(")", "_") \
                               .replace("\n", "_").replace("\t", "_").replace("=", "_")
        test_case_df = test_case_df.withColumnRenamed(col_name, new_col_name)

    test_case_df_final = test_case_df.withColumn("ingestion_timestamp", lit(run_timestamp)) \
                                     .withColumn("period", date_format(current_timestamp(), "yyyy-MM")) \
                                     .withColumn("tc_key", sha2(col("ingestion_timestamp").cast("string"), 256))
    
    test_case_df_final.write.format("delta") \
                            .option("mergeSchema", "true") \
                            .mode("append") \
                            .saveAsTable("logging_db.test_case_results")
    
    display(f"Successfully appended {test_case_df_final.count()} records to logging_db.test_case_results.")
    display(test_case_df_final)

adf_run_records = []
adf_details_dict = combined_output.get('adf_run_details', {})

for file_name, details in adf_details_dict.items():
    if details:
        adf_run_records.append({
            "File_Name": file_name,
            "run_id": details.get('id','N/A'),
            "run_status": details.get('run_status','N/A'),
            "Error Message": details.get('message','N/A'),
            "run_duration_min": float(details.get('run_duration_min', 0.0)),
            "environment": env
        })
    else:
        adf_run_records.append({
            "File_Name": file_name,
            "run_id": 'N/A',
            "run_status": 'N/A',
            "Error Message": 'N/A',
            "run_duration_min":'N/A',
            "environment": env
        })

if adf_run_records:
    adf_run_df = spark.createDataFrame(adf_run_records)

    for col_name in adf_run_df.columns:
        new_col_name = col_name.replace(" ", "_").replace(";", "_").replace("{", "_") \
                               .replace("}", "_").replace("(", "_").replace(")", "_") \
                               .replace("\n", "_").replace("\t", "_").replace("=", "_")
        adf_run_df = adf_run_df.withColumnRenamed(col_name, new_col_name)

    adf_run_df_final = adf_run_df.withColumn("ingestion_timestamp", lit(run_timestamp)) \
                                 .withColumn("period", date_format(current_timestamp(), "yyyy-MM")) \
                                 .withColumn("run_duration_min", col("run_duration_min").cast(DoubleType())) \
                                 .withColumn("ar_key", sha2(col("ingestion_timestamp").cast("string"), 256))
    adf_run_df_final = adf_run_df_final.select("File_Name", "run_id", "run_status", "Error_Message", "run_duration_min", "environment", "ingestion_timestamp", "period", "ar_key")
    
    adf_run_df_final.write.format("delta") \
                          .option("mergeSchema", "true") \
                          .mode("append") \
                          .saveAsTable("logging_db.adf_run_details")
    
    display(f"Successfully appended {adf_run_df_final.count()} records to logging_db.adf_run_details.")
    display(adf_run_df_final)

In [0]:
dbutils.notebook.exit(json.dumps(combined_output))